# **1) Import all packages**

In [2]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import math
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split 
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
tf.random.set_seed(42)
seed(42)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# **2) Get Dataset**

In [3]:
df = pd.read_excel('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/default of credit card clients.xls', index_col=0)

In [4]:
print(df.head())
print(df.shape)
print(df.shape)

           X1   X2         X3  ...       X22       X23                           Y
ID  LIMIT_BAL  SEX  EDUCATION  ...  PAY_AMT5  PAY_AMT6  default payment next month
1       20000    2          2  ...         0         0                           1
2      120000    2          2  ...         0      2000                           1
3       90000    2          2  ...      1000      5000                           0
4       50000    2          2  ...      1069      1000                           0

[5 rows x 24 columns]
(30001, 24)
(30001, 24)


In [5]:
print(df.columns)

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'Y'],
      dtype='object')


# **3) SMOTE Oversample Dataset**

In [6]:
df['Y'].value_counts()

0                             23364
1                              6636
default payment next month        1
Name: Y, dtype: int64

In [7]:
 x = df.isna().sum()
 print(x)

X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
X16    0
X17    0
X18    0
X19    0
X20    0
X21    0
X22    0
X23    0
Y      0
dtype: int64


In [8]:
#Get labels
labels = df.pop('Y')

In [9]:
labels_array = labels[1:len(labels)].to_numpy() 
print(labels_array)
labelsv2 = np.vstack(labels_array)
print(labelsv2)
labels[1:len(labels)].value_counts(normalize=True) * 100

[1 1 0 ... 1 1 1]
[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


0    77.88
1    22.12
Name: Y, dtype: float64

In [10]:
#Split data
X_train, X_test, y_train, y_test =  train_test_split(df[1:len(df)], labelsv2, train_size=0.85, test_size=0.15, random_state=42)

In [11]:
print(X_train.shape)
print(y_train.shape)

(25500, 23)
(25500, 1)


In [12]:
#Scale Data
# Training Data
X_train = MinMaxScaler().fit_transform(X_train)

# Testing Data
X_test = MinMaxScaler().fit_transform(X_test)

In [13]:
# Synthesize minority class datapoints using SMOTE
sm = SMOTE(random_state=42, sampling_strategy='minority')
smote_x_train, smote_y_train = sm.fit_resample(X_train, y_train.ravel())
smote_x_test, smote_y_test = sm.fit_resample(X_test, y_test.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [14]:
X_train = smote_x_train
y_train = smote_y_train
X_test = smote_x_test
y_test = smote_y_test

# **4) Vanilla Decision Tree**

In [44]:
#Create decision tree for specified depth of 5 and 10 
model = DecisionTreeClassifier(max_depth=10,random_state=16)
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=16, splitter='best')

In [45]:
#Get predictions of vanilla DT
res_pred = model.predict(X_test)

In [46]:
#Get various scores
acc = accuracy_score(y_test, res_pred)
rec = recall_score(y_test, res_pred,average='macro')
prec = precision_score(y_test, res_pred,average='macro')
auc = roc_auc_score(y_test,res_pred, average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.5058621675722047
Recall:  0.5058621675722048
Precision: 0.563327819966212
AUC:  0.5534744066342578


In [18]:
#Save for later use
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_train.npy",smote_x_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_trainlabels.npy",smote_y_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_test.npy",smote_x_test)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_testlabels.npy",smote_y_test)

In [19]:
#Open if needed to re-run
#X_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_train.npy")
#y_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_trainlabels.npy")
#X_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_test.npy")
#y_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/SMOTE_testlabels.npy")

# **5) Baseline Model**

In [20]:
log_X_train = X_train
log_y_train = y_train
log_X_test = X_test
log_y_test = y_test

In [21]:
#Create Logistic Regression Model
logreg = LogisticRegression(max_iter= 3000, random_state=16)
logreg.fit(log_X_train,log_y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=16, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
#Get Baseline score
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
print(logit_roc_auc)

0.6436945953674578


# **6) Built Neural Network**

In [23]:
print(X_train.shape)
print(X_train)
print(y_train.shape)
X_train = np.asarray(X_train).astype('float32')

(39734, 23)
[[2.02020202e-02 1.00000000e+00 3.33333333e-01 ... 5.40257649e-03
  2.68680441e-03 0.00000000e+00]
 [1.61616162e-01 0.00000000e+00 6.66666667e-01 ... 0.00000000e+00
  1.14927707e-02 1.13821107e-02]
 [3.33333333e-01 0.00000000e+00 1.66666667e-01 ... 3.09178744e-03
  2.14545787e-02 3.54742451e-01]
 ...
 [3.20413901e-02 1.00000000e+00 1.66666667e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.37980446e-01 1.00000000e+00 1.66666667e-01 ... 0.00000000e+00
  0.00000000e+00 2.09341647e-04]
 [1.12317916e-01 0.00000000e+00 3.33333333e-01 ... 1.77133655e-03
  3.13240460e-03 2.80775372e-03]]
(39734,)


In [24]:
print(X_test.shape)
print(y_test.shape)
print(y_test)
X_test = np.asarray(X_test).astype('float32')

(6994, 23)
(6994,)
[0 0 0 ... 1 1 1]


In [25]:
#Prepare Labels
y_train = np.array(keras.utils.to_categorical(y_train))
y_test = np.array(keras.utils.to_categorical(y_test))
print(y_train.shape)
print(y_train[:10])

(39734, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [26]:
#Metrics for Neural Network
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

val_auc = tf.keras.metrics.AUC()

In [27]:
#Function to built model
def build_model():
  inputs = layers.Input(shape=(23))

  x = layers.Dense(23,activation='relu',name='fc1')(inputs)

  x = layers.Dense(20,activation='relu',name='fc2')(x)

  x = layers.Dense(10,activation='relu',name='fc3')(x)

  x = layers.Dense(2,name='logits')(x)

  preds = layers.Activation('softmax',name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model
model = build_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
____________________________________________________

In [28]:
#Saving best performing model
filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Upsampling/Upsampling Models/Model_retestV2.hdf5"

In [29]:
#Set-up filepath and ModelCheckepoint with Earlystopping to save best performing moddel based on Validation AUC
model_checkpoint = ModelCheckpoint(filepath,  monitor='val_auc', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_auc', mode='max', verbose=0, patience=5)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC(name='auc')])

In [30]:
#Set epochs and bach size, and train model 
num_epoch = 100
batch_size = 64
model1 = model.fit(x=X_train, y=y_train, epochs=num_epoch, batch_size=batch_size, validation_split=0.15,callbacks=[model_checkpoint,es])

Epoch 1/100
528/528 [==============================] - 4s 7ms/step - loss: 0.6084 - acc: 0.6916 - f1_m: 0.6915 - precision_m: 0.6915 - recall_m: 0.6915 - auc: 0.7334 - val_loss: 0.7659 - val_acc: 0.4444 - val_f1_m: 0.4444 - val_precision_m: 0.4444 - val_recall_m: 0.4444 - val_auc: 0.5467
Epoch 2/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5738 - acc: 0.7226 - f1_m: 0.7227 - precision_m: 0.7227 - recall_m: 0.7227 - auc: 0.7707 - val_loss: 0.9043 - val_acc: 0.4127 - val_f1_m: 0.4130 - val_precision_m: 0.4130 - val_recall_m: 0.4130 - val_auc: 0.4459
Epoch 3/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5678 - acc: 0.7248 - f1_m: 0.7248 - precision_m: 0.7248 - recall_m: 0.7248 - auc: 0.7775 - val_loss: 0.7496 - val_acc: 0.4716 - val_f1_m: 0.4713 - val_precision_m: 0.4713 - val_recall_m: 0.4713 - val_auc: 0.6220
Epoch 4/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5645 - acc: 0.7265 - f1_m: 0.7265 - precision_m: 0.7265 

In [31]:
#Train Performance
prediction_model = tf.keras.models.load_model(filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_train, y_train, verbose=0, batch_size=64)

print('Train Accuracy:',accuracy)
print('Train Recall:',recall)
print('Train Precision:',precision)
print('Train F1:',f1_score)
print('Train AUC:',auc)
prediction_model.summary()

Train Accuracy: 0.7048623561859131
Train Recall: 0.7048200964927673
Train Precision: 0.7048200964927673
Train F1: 0.7048200964927673
Train AUC: 0.7809620499610901
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)   

In [32]:
#Test Performance
#filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/CC Models/ModelV4.hdf5"
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

print('Test Accuracy:',accuracy)
print('Test Recall:',recall)
print('Test Precision:',precision)
print('Test F1:',f1_score)
print('Test AUC:',auc)
prediction_model.summary()

Test Accuracy: 0.6844438314437866
Test Recall: 0.6839646697044373
Test Precision: 0.6839646697044373
Test F1: 0.6839646100997925
Test AUC: 0.7515403628349304
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)        

In [33]:
#Best Performance
best_filepath =  "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Upsampling/Upsampling Models/Model_retestV2.hdf5"
best_model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = best_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

#print('Best Accuracy:',accuracy)
#print('Best Recall:',recall)
#print('Best Precision:',precision)
#print('Best F1:',f1_score)
print('Best AUC:',auc)
best_model.summary()

Best AUC: 0.7515403628349304
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
_______________________

# **7) Generate Soft Targets**

In [34]:
#Load the best model
model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m}) 

In [35]:
#Compile it again
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC()])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
____________________________________________________

In [36]:
#Remove last softmax layer and predict again
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(X_train)

In [37]:
#Get the logits
soft_targets = model_logits

In [38]:
#Save if needed for later use
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_Logits.npy", model_logits)

# **8) Knowledge Distillation Decision Tree**


In [62]:
#Set up Decision Tree for Knowledge Distillation at given depth 
model = DecisionTreeRegressor(max_depth=5,random_state=16)

In [63]:
#Train Decision Tree
model.fit(X_train,soft_targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=16, splitter='best')

In [64]:
#Get predictions
res_pred = model.predict(X_test)

In [65]:
# Apply softmax function to output of Decision Tree
pred_argmax = np.zeros_like(res_pred)
pred_argmax[np.arange(len(res_pred)), res_pred.argmax(1)] = 1 

In [66]:
#Get scores of student decision tree
acc = accuracy_score(pred_argmax,y_test)
rec = recall_score(y_test, pred_argmax,average='macro')
prec = precision_score(y_test, pred_argmax,average='macro')
auc = roc_auc_score(y_test, pred_argmax,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.6802973977695167
Recall:  0.6802973977695167
Precision: 0.7167437049115899
AUC:  0.6802973977695167
